# Multiclase, Multilabel, MultiSalida

En sklearn los clasificadores están ordenados de la siguiente manera:

- Multiclase: Para tareas de clasificación en donde tenemos más de dos clases, pero cada instancia solo puede pertenecer a una. Por ejemplo clasificar imágenes en coches, avión o tren. Una sola imágen no puede ser avión y tren a la vez.

- Multilabel: Para tareas de clasificación donde tenemos más de dos clases, cada instancia puede pertenece tener un conjunto de clases simultaneamente. Por ejemplo un texto puede pertenecer a la clase política y a la clase religión simultaneamente.

- Multilasalida:
    - Mutiregresion: Cada instancia o cada ejemplo puede tener asociadas varias variables objetivo de tipo numérico. 
    
    - Multiclasificación: Es una generalización de multilabel y multiclase. Cada instancia está asociado con un conjunto de variables de salida, que puede tomar cada una de ellas varios valores. Por ejemplo una clase puede ser objeto: coche, avión, tren y otra el color: rojo, azul, amarillo. Este es el caso del juego, la variable movimiento puede ser derecha, izquierda, no-moverse y la de disparo: si/no.

Todos los clasificadores de Sklearn tienen capacidad multiclase. Pero solo árboles, random forest y vecinos más cercanos tienen capacidad multisalida.

MultiOutputClassifier puede convertir cualquier clasificador en multisalida.


## Ejemplos de salidas

### Multiclase

In [1]:
from sklearn import datasets

iris = datasets.load_iris()
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

### Multilabel

In [2]:
from sklearn.datasets import make_multilabel_classification
X, y1 = make_multilabel_classification(n_samples=10, n_features=100, n_classes=3, random_state=1)
y1

array([[0, 1, 0],
       [1, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 1, 0],
       [0, 1, 0],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

### MultiSalida

En el caso del videojuego.
Dirección podría valer -1, 0, 1   (izd, nada, der) y disparar solo 0, 1

In [3]:
from sklearn.datasets import make_classification
from sklearn.utils import shuffle
import numpy as np

X, y1 = make_classification(n_samples=10, n_features=100, n_informative=30, n_classes=3, random_state=1)
y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T

Y

array([[2, 2, 0],
       [1, 2, 1],
       [2, 1, 0],
       [0, 0, 2],
       [0, 2, 1],
       [0, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [0, 0, 2],
       [2, 0, 0]])

Entrenar y predecir se hace exactamente igual que con multiclase
**fit** y **predict**

In [9]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

X, y1 = make_classification(n_samples=100, n_features=20, n_informative=10, n_classes=4, random_state=1)
y2 = shuffle(y1, random_state=1)
y3 = shuffle(y1, random_state=2)
Y = np.vstack((y1, y2, y3)).T
n_samples, n_features = X.shape # 10,100
n_outputs = Y.shape[1] # 3
n_classes = 3
forest = RandomForestClassifier(n_estimators=100, random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

# Random Forest ya es multi target, pero lo pruebo de las dos maneras
Ypred1 = multi_target_forest.fit(X, Y).predict([X[0]])
Y.shape

(100, 3)

In [5]:
Ypred2 = forest.fit(X, Y).predict(X)

## Medidas de evaluación

Por el momento no hay medidas multioutput en sklearn (oficialmente)

Una opción es medir el accuracy (porcentaje de acierto) de cada variable de salida individualmente y otra es implementar alguna medida nosotros.

Hay gente que ha hecho eso.
https://github.com/scikit-learn/scikit-learn/pull/3681/commits



Obtengo las predicciones mediante validación cruzada.

In [6]:
from sklearn.model_selection import cross_val_predict

predicted = cross_val_predict(forest, X, Y, cv=5)



Me quedo solo con la primera clase y comparo la predicción con la real para sacar el porcentaje de acierto

In [7]:
from sklearn import metrics
primeraClaseP = predicted[:,0] # accedo a la primera columna, todos los resultados de la clase1
primeraClaseR = Y[:,0]

print(primeraClaseP.astype(int)[:10]) # primeras 10 predicciones
print(primeraClaseR[:10]) # primeras 10 clases reales

metrics.accuracy_score(primeraClaseR, primeraClaseP)

[1 1 3 3 2 2 3 1 1 1]
[0 2 2 3 1 0 3 0 2 2]


0.47999999999999998

# Histograma

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame({"A": [9, 4, 2, 1, 2,3,5], "B": [12, 7, 5, 4, 2,3,5]})
C = pd.DataFrame({"C": [1, 1, 0, 1, 0,1,1]})
df

,A,B
0,9,12
1,4,7
2,2,5
3,1,4
4,2,2
5,3,3
6,5,5


In [4]:
dfAnterior = df.shift(1)
dfAnterior.rename(columns=lambda x: x+"-1", inplace=True)

dfAnterior2 = df.shift(2)
dfAnterior2.rename(columns=lambda x: x+"-2", inplace=True)
dfAnterior2

,A-2,B-2
0,NaN,NaN
1,NaN,NaN
2,9.0,12.0
3,4.0,7.0
4,2.0,5.0
5,1.0,4.0
6,2.0,2.0


In [5]:
# concat, concatena listas de 2, 3 dataframes o el número que se quiera. Así que se puede parametriz
pd.concat([df,dfAnterior,dfAnterior2,C],axis=1)

,A,B,A-1,B-1,A-2,B-2,C
0,9,12,NaN,NaN,NaN,NaN,1
1,4,7,9.0,12.0,NaN,NaN,1
2,2,5,4.0,7.0,9.0,12.0,0
3,1,4,2.0,5.0,4.0,7.0,1
4,2,2,1.0,4.0,2.0,5.0,0
5,3,3,2.0,2.0,1.0,4.0,1
6,5,5,3.0,3.0,2.0,2.0,1


## Histograma


con

```Python
dfHist["Bin"] = dfHist["A"]//2.5
```

Estoy creando una variable, con nombre Bin cuyo valor es la banda del histograma donde cae el valor A (primera banda (0), segunda 1, ...). 

In [6]:

dfHist = df.copy()
dfHist["Bin"] = dfHist["A"]//2.5
dfHist


,A,B,Bin
0,9,12,3.0
1,4,7,1.0
2,2,5,0.0
3,1,4,0.0
4,2,2,0.0
5,3,3,1.0
6,5,5,2.0


In [7]:
dfHist[['A',"Bin"]]

,A,Bin
0,9,3.0
1,4,1.0
2,2,0.0
3,1,0.0
4,2,0.0
5,3,1.0
6,5,2.0


Luego puedo quedarme con la suma de los valores que hay en cada bin

In [11]:
dfHist[['A','B',"Bin"]].groupby('Bin').sum()


,A,B
Bin,,
0.0,5,11
1.0,7,10
2.0,5,5
3.0,9,12
